# PARTIE 3 - Application de Prédiction

## Qui sera le vainqueur ? 

Cette application utilise le modèle d'apprentissage entraîné pour prédire le vainqueur d'un combat entre deux Pokémons.

### Fonctionnement :
1. Choisir deux Pokémons par leur numéro
2. Le modèle prédit le pourcentage de victoire de chaque Pokémon
3. Le Pokémon avec le pourcentage le plus élevé est déclaré vainqueur !

### Utilisation du modèle sauvegardé :
Nous allons charger le modèle `modele_pokemon.mod` créé lors de l'entraînement.

## 1. Import des modules nécessaires

In [1]:
# Module de lecture de fichiers CSV
import csv
# Module de chargement du modèle d'apprentissage
import joblib
# Module pandas pour faciliter la manipulation
import pandas as pd

print("Modules importés avec succès !")

Modules importés avec succès !


## 2. Fonction de recherche d'informations sur un Pokémon

Cette fonction cherche les informations d'un Pokémon dans le Pokédex à partir de son numéro.

In [10]:
def rechercheInformationsPokemon(numPokemon, Pokedex):
    """
    Recherche les informations d'un Pokémon dans le Pokédex
    
    Paramètres:
    - numPokemon: numéro du Pokémon à rechercher
    - Pokedex: liste contenant tous les Pokémons
    
    Retourne:
    - Liste contenant: [NUMERO, NOM, POINTS_DE_VIE, NIVEAU_ATTAQUE, NIVEAU_DEFENSE,
                        NIVEAU_ATTAQUE_SPECIALE, NIVEAU_DEFENSE_SPECIALE, VITESSE, GENERATION, LEGENDAIRE]
    """
    infosPokemon = []
    
    for pokemon in Pokedex:
        if int(pokemon[0]) == numPokemon:
            # Extraction des informations nécessaires pour la prédiction
            # Colonnes: 0=NUMERO, 1=NOM, 4=POINTS_DE_VIE, 5=NIVEAU_ATTAQUE, 6=NIVEAU_DEFENSE,
            #           7=NIVEAU_ATTAQUE_SPECIALE, 8=NIVEAU_DEFENSE_SPECIALE, 9=VITESSE, 10=GENERATION, 11=LEGENDAIRE
            infosPokemon = [
                pokemon[0],  # NUMERO
                pokemon[1],  # NOM
                pokemon[4],  # POINTS_DE_VIE
                pokemon[5],  # NIVEAU_ATTAQUE
                pokemon[6],  # NIVEAU_DEFENSE
                pokemon[7],  # NIVEAU_ATTAQUE_SPECIALE
                pokemon[8],  # NIVEAU_DEFENSE_SPECIALE
                pokemon[9],  # VITESSE
                pokemon[10], # GENERATION
                pokemon[11]  # LEGENDAIRE
            ]
            break
    
    return infosPokemon

print("Fonction rechercheInformationsPokemon définie")

Fonction rechercheInformationsPokemon définie


## 3. Fonction de prédiction du vainqueur

Cette fonction prédit qui des deux Pokémons sera le vainqueur du combat.

In [13]:
def prediction(numeroPokemon1, numeroPokemon2, Pokedex):
    """
    Prédit le vainqueur d'un combat entre deux Pokémons
    
    Paramètres:
    - numeroPokemon1: numéro du premier Pokémon
    - numeroPokemon2: numéro du second Pokémon
    - Pokedex: liste contenant tous les Pokémons
    """
    
    # Recherche des informations des deux Pokémons
    pokemon1 = rechercheInformationsPokemon(numeroPokemon1, Pokedex)
    pokemon2 = rechercheInformationsPokemon(numeroPokemon2, Pokedex)
    
    if not pokemon1 or not pokemon2:
        print("Erreur : Un ou plusieurs Pokémons n'ont pas été trouvés")
        return
    
    # Chargement du modèle de prédiction
    modele_prediction = joblib.load('modele/modele_pokemon.mod')
    
    # Préparation des données pour la prédiction
    # On utilise les colonnes 2 à 9 : POINTS_DE_VIE, NIVEAU_ATTAQUE, NIVEAU_DEFENSE,
    # NIVEAU_ATTAQUE_SPECIALE, NIVEAU_DEFENSE_SPECIALE, VITESSE, GENERATION, LEGENDAIRE
    
    # Conversion de LEGENDAIRE : VRAI -> 1, FAUX -> 0
    legendaire1 = 1 if pokemon1[9] == "VRAI" else 0
    legendaire2 = 1 if pokemon2[9] == "VRAI" else 0
    
    donnees_pokemon1 = [[
        float(pokemon1[2]),  # POINTS_DE_VIE
        float(pokemon1[3]),  # NIVEAU_ATTAQUE
        float(pokemon1[4]),  # NIVEAU_DEFENSE
        float(pokemon1[5]),  # NIVEAU_ATTAQUE_SPECIALE
        float(pokemon1[6]),  # NIVEAU_DEFENSE_SPECIALE
        float(pokemon1[7]),  # VITESSE
        float(pokemon1[8]),  # GENERATION
        float(legendaire1)   # LEGENDAIRE (converti en 0/1)
    ]]
    
    donnees_pokemon2 = [[
        float(pokemon2[2]),  # POINTS_DE_VIE
        float(pokemon2[3]),  # NIVEAU_ATTAQUE
        float(pokemon2[4]),  # NIVEAU_DEFENSE
        float(pokemon2[5]),  # NIVEAU_ATTAQUE_SPECIALE
        float(pokemon2[6]),  # NIVEAU_DEFENSE_SPECIALE
        float(pokemon2[7]),  # VITESSE
        float(pokemon2[8]),  # GENERATION
        float(legendaire2)   # LEGENDAIRE (converti en 0/1)
    ]]
    
    # Prédictions
    prediction_Pokemon_1 = modele_prediction.predict(donnees_pokemon1)
    prediction_Pokemon_2 = modele_prediction.predict(donnees_pokemon2)
    
    # Affichage des résultats
    print("=" * 80)
    print(f"COMBAT OPPOSANT : (#{numeroPokemon1}) {pokemon1[1]} à (#{numeroPokemon2}) {pokemon2[1]}")
    print("=" * 80)
    print(f"\nPRÉDICTIONS :")
    print(f"   {pokemon1[1]}: {prediction_Pokemon_1[0]:.4f} ({prediction_Pokemon_1[0]*100:.2f}%)")
    print(f"   {pokemon2[1]}: {prediction_Pokemon_2[0]:.4f} ({prediction_Pokemon_2[0]*100:.2f}%)")
    print()
    
    # Détermination du vainqueur
    if prediction_Pokemon_1[0] > prediction_Pokemon_2[0]:
        print(f"{pokemon1[1].upper()} EST LE VAINQUEUR !")
        difference = (prediction_Pokemon_1[0] - prediction_Pokemon_2[0]) * 100
        print(f"   Avantage de {difference:.2f}% de points de victoire")
    else:
        print(f"{pokemon2[1].upper()} EST LE VAINQUEUR !")
        difference = (prediction_Pokemon_2[0] - prediction_Pokemon_1[0]) * 100
        print(f"   Avantage de {difference:.2f}% de points de victoire")
    
    print("=" * 80)

print("Fonction prediction définie")

Fonction prediction définie


## 4. Exemple de prédiction

Testons notre application avec un combat !

In [14]:
# Chargement du Pokédex et lancement d'un combat
with open("datas/pokedex.csv", newline='') as csvfile:
    pokedex = csv.reader(csvfile)
    next(pokedex)  # Ignorer la ligne d'en-tête
    pokedex_list = list(pokedex)  # Convertir en liste pour réutilisation

# Exemple 1: Combat entre Mangriff (#368) et Crapustule (#598)
print("EXEMPLE 1:")
prediction(368, 598, pokedex_list)

EXEMPLE 1:
COMBAT OPPOSANT : (#368) Mangriff à (#598) Crapustule

PRÉDICTIONS :
   Mangriff: 0.7133 (71.33%)
   Crapustule: 0.5890 (58.90%)

MANGRIFF EST LE VAINQUEUR !
   Avantage de 12.43% de points de victoire
COMBAT OPPOSANT : (#368) Mangriff à (#598) Crapustule

PRÉDICTIONS :
   Mangriff: 0.7133 (71.33%)
   Crapustule: 0.5890 (58.90%)

MANGRIFF EST LE VAINQUEUR !
   Avantage de 12.43% de points de victoire


## 5. Vérification avec des combats du fichier combats.csv

Vérifions nos prédictions avec les résultats réels du fichier combats.csv.

In [15]:
# Charger quelques combats du fichier pour vérifier nos prédictions
combats_test = [
    (155, 321, 155),  # Combat 1
    (101, 583, 583),  # Combat 2
    (404, 32, 32)     # Combat 3
]

print("\n" + "VÉRIFICATION AVEC DES COMBATS RÉELS ".center(80, "="))
print()

for i, (poke1, poke2, gagnant_reel) in enumerate(combats_test, 1):
    print(f"\n{'='*80}")
    print(f"TEST #{i} - Vainqueur réel du combat : Pokémon #{gagnant_reel}")
    print(f"{'='*80}")
    prediction(poke1, poke2, pokedex_list)
    print()


======================VÉRIFICATION AVEC DES COMBATS RÉELS ======================


TEST #1 - Vainqueur réel du combat : Pokémon #155
COMBAT OPPOSANT : (#155) Mega Ptera à (#321) Makuhita

PRÉDICTIONS :
   Mega Ptera: 0.9735 (97.35%)
   Makuhita: 0.1039 (10.39%)

MEGA PTERA EST LE VAINQUEUR !
   Avantage de 86.95% de points de victoire


TEST #2 - Vainqueur réel du combat : Pokémon #583
COMBAT OPPOSANT : (#155) Mega Ptera à (#321) Makuhita

PRÉDICTIONS :
   Mega Ptera: 0.9735 (97.35%)
   Makuhita: 0.1039 (10.39%)

MEGA PTERA EST LE VAINQUEUR !
   Avantage de 86.95% de points de victoire


TEST #2 - Vainqueur réel du combat : Pokémon #583
COMBAT OPPOSANT : (#101) Spectrum à (#583) Zeblitz

PRÉDICTIONS :
   Spectrum: 0.6995 (69.95%)
   Zeblitz: 0.9179 (91.79%)

ZEBLITZ EST LE VAINQUEUR !
   Avantage de 21.85% de points de victoire


TEST #3 - Vainqueur réel du combat : Pokémon #32
COMBAT OPPOSANT : (#101) Spectrum à (#583) Zeblitz

PRÉDICTIONS :
   Spectrum: 0.6995 (69.95%)
   Zeblitz: 0